In [1]:
#Создаем соединение
import psycopg2
import getpass

conn=psycopg2.connect(user='postgres',
                     database='postgres',
                     host='localhost',
                     port='5432',
                     password=getpass.getpass('Insert password'))

Insert password········


In [2]:
# Получение данных из БД PostgreSQL, с которыми будем дальше работать, при помощи запроса SQL. 
# Запрашиваем данные из таблиц bond_description и quotes, объединенных по общему полю "ISIN"
# Нам необходимо все, что потребуется для создания матрицы платежей: размер купона, даты выплат купона, номинал,
# дата погашения и проч (см.результат ниже)
# При этом нужно исключить облигации, которые погашены, размещаются в данный момент и по которым случился дефолт.
# + по рекомендации feedback к данному дз добавлено поле типа ценной бумаги в запрос из БД
import pandas as pd
query_bonds="SELECT DISTINCT bond_description.\"ISINCODE\",bond_description.\"ISSUERNAME\", bond_description.\"HAVEOFFER\", bond_description.\"COUPONTYPE\", bond_description.\"SECURITYTYPE\", quotes.\"BUYBACKDATE\", quotes.\"CPN\", quotes.\"CPN_DATE\", quotes.\"CPN_PERIOD\", quotes.\"FACEVALUE\", quotes.\"MAT_DATE\" FROM bond_description INNER JOIN quotes ON bond_description.\"ISINCODE\"=quotes.\"ISIN\" WHERE \"STATUS\"!='Погашен' and \"STATUS\"!='Размещается' and \"HAVEDEFAULT\"='0'"
bonds_desc=pd.read_sql_query(query_bonds,conn)

# Проверяем размерность таблицы

bonds_desc.shape

(4468, 11)

In [3]:
# Выводим первые 10 строк для проверки
bonds_desc.head(10)

,ISINCODE,ISSUERNAME,HAVEOFFER,COUPONTYPE,SECURITYTYPE,BUYBACKDATE,CPN,CPN_DATE,CPN_PERIOD,FACEVALUE,MAT_DATE
0,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,4.99,2012-08-23,182.0,1000.0,2016-02-18
1,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2011-02-24,182.0,1000.0,2016-02-18
2,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2011-08-25,182.0,1000.0,2016-02-18
3,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2012-02-23,182.0,1000.0,2016-02-18
4,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2012-08-23,182.0,1000.0,2016-02-18
5,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2009-08-27,NaN,1000.0,2016-02-18
6,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2010-02-25,NaN,1000.0,2016-02-18
7,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2010-08-26,NaN,1000.0,2016-02-18
8,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2011-02-24,NaN,1000.0,2016-02-18
9,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,None,NaN,1000.0,2016-02-18


In [5]:
# Отбираем из запрошенной из БД таблицы данные для облигаций с постоянным купоном

fixed_coup=bonds_desc[['ISINCODE','CPN','CPN_DATE','MAT_DATE','FACEVALUE']][(bonds_desc['COUPONTYPE']=='Постоянный')|(bonds_desc['COUPONTYPE']=='Фиксированный')]
fixed_coup['CPN'].fillna(0, inplace=True)
fixed_coup.head(10)

,ISINCODE,CPN,CPN_DATE,MAT_DATE,FACEVALUE
77,RU000A0JNYN1,34.9,2013-06-11,2022-06-11,1000.0
78,RU000A0JNYN1,34.9,2013-12-11,2022-06-11,1000.0
79,RU000A0JNYN1,34.9,2014-06-11,2022-06-11,1000.0
80,RU000A0JNYN1,34.9,2014-12-11,2022-06-11,1000.0
81,RU000A0JNYN1,34.9,2015-06-11,2022-06-11,1000.0
82,RU000A0JNYN1,34.9,2015-12-11,2022-06-11,1000.0
83,RU000A0JNYN1,35.1,2012-12-11,2022-06-11,1000.0
84,RU000A0JNYN1,35.1,2013-06-11,2022-06-11,1000.0
85,RU000A0JNYN1,35.1,2013-12-11,2022-06-11,1000.0
86,RU000A0JNYN1,35.1,2014-06-11,2022-06-11,1000.0


In [8]:
# Считаем ден поток по облигациям с пост или фикс купоном
import numpy as np
fixed_coup['CF']=np.where(fixed_coup['CPN_DATE']==fixed_coup['MAT_DATE'], fixed_coup['CPN']+fixed_coup['FACEVALUE'], fixed_coup['CPN'])
fixed_coup
# Для проверки (Код ниже проверяет, правильно ли заполнились ячейки по условию выше )

fixed_coup.loc[fixed_coup['CF'] != (fixed_coup['CPN']+fixed_coup['FACEVALUE'])]

,ISINCODE,CPN,CPN_DATE,MAT_DATE,FACEVALUE,CF
77,RU000A0JNYN1,34.9000,2013-06-11,2022-06-11,1000.0,34.9000
78,RU000A0JNYN1,34.9000,2013-12-11,2022-06-11,1000.0,34.9000
79,RU000A0JNYN1,34.9000,2014-06-11,2022-06-11,1000.0,34.9000
80,RU000A0JNYN1,34.9000,2014-12-11,2022-06-11,1000.0,34.9000
81,RU000A0JNYN1,34.9000,2015-06-11,2022-06-11,1000.0,34.9000
82,RU000A0JNYN1,34.9000,2015-12-11,2022-06-11,1000.0,34.9000
83,RU000A0JNYN1,35.1000,2012-12-11,2022-06-11,1000.0,35.1000
84,RU000A0JNYN1,35.1000,2013-06-11,2022-06-11,1000.0,35.1000
85,RU000A0JNYN1,35.1000,2013-12-11,2022-06-11,1000.0,35.1000
86,RU000A0JNYN1,35.1000,2014-06-11,2022-06-11,1000.0,35.1000


In [10]:
# Для проверки (вторая часть условия)
fixed_coup.loc[fixed_coup['CF'] != fixed_coup['CPN']]

,ISINCODE,CPN,CPN_DATE,MAT_DATE,FACEVALUE,CF
958,RU000A0JR530,21.0800,2015-11-25,2015-11-25,500.0,521.0800
959,RU000A0JR530,21.4200,2015-11-25,2015-11-25,500.0,521.4200
960,RU000A0JR530,21.4247,2015-11-25,2015-11-25,500.0,521.4247
981,RU000A0JR6K5,9.2700,2015-12-17,2015-12-17,400.0,409.2700
982,RU000A0JR6K5,9.2745,2015-12-17,2015-12-17,400.0,409.2745
1013,RU000A0JR852,42.3800,2016-02-02,2016-02-02,1000.0,1042.3800
1014,RU000A0JR852,42.3836,2016-02-02,2016-02-02,1000.0,1042.3836
1383,RU000A0JRW93,43.6400,2016-01-31,2016-01-31,1000.0,1043.6400
1388,RU000A0JRW93,44.3600,2016-01-31,2016-01-31,1000.0,1044.3600
1389,RU000A0JRW93,44.3616,2016-01-31,2016-01-31,1000.0,1044.3616


In [14]:
# Отбираем нужные для матрицы поля
fcoup_formatrix=fixed_coup[['ISINCODE','CPN_DATE','CF']]
fcoup_formatrix.head(10)

,ISINCODE,CPN_DATE,CF
77,RU000A0JNYN1,2013-06-11,34.9
78,RU000A0JNYN1,2013-12-11,34.9
79,RU000A0JNYN1,2014-06-11,34.9
80,RU000A0JNYN1,2014-12-11,34.9
81,RU000A0JNYN1,2015-06-11,34.9
82,RU000A0JNYN1,2015-12-11,34.9
83,RU000A0JNYN1,2012-12-11,35.1
84,RU000A0JNYN1,2013-06-11,35.1
85,RU000A0JNYN1,2013-12-11,35.1
86,RU000A0JNYN1,2014-06-11,35.1


In [15]:
# Формируем матрицу для облигаций с фиксированным или постоянным купоном
fcoup_formatrix=fcoup_formatrix.pivot_table(index=['ISINCODE'],
                      columns='CPN_DATE',
                     values='CF').reset_index().rename_axis(None, axis=1)
fcoup_formatrix=fcoup_formatrix.fillna(0)
fcoup_formatrix

,ISINCODE,2009-06-11,2009-06-14,2009-09-22,2009-12-11,2009-12-14,2009-12-28,2010-03-22,2010-06-11,2010-06-14,...,2016-03-25,2016-03-29,2016-04-04,2016-04-06,2016-04-07,2016-04-08,2016-04-14,2016-04-19,2016-04-22,2016-04-27
0,RU000A0JNYN1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0
1,RU000A0JNYP6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0
2,RU000A0JPZ50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0
3,RU000A0JQ6S9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0
4,RU000A0JR4H6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0
5,RU000A0JR530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0
6,RU000A0JR6K5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0
7,RU000A0JR852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0
8,RU000A0JRGR0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0
9,RU000A0JRJN3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.00000,0.00000,0.0000,0.0000,0.0


In [60]:
# Функция для облигаций с постоянным и фиксированным купоном
# Здесь просто собираем все проделанные вручную выше действия под одной функцией
def my_fun2(issuername, cpndate, conn):
    import pandas as pd
    query_bonds="SELECT DISTINCT bond_description.\"ISINCODE\",bond_description.\"ISSUERNAME\", "\
    "bond_description.\"HAVEOFFER\", bond_description.\"COUPONTYPE\", bond_description.\"SECURITYTYPE\","\
    "quotes.\"BUYBACKDATE\", quotes.\"CPN\", quotes.\"CPN_DATE\", quotes.\"CPN_PERIOD\", quotes.\"FACEVALUE\","\
    "quotes.\"MAT_DATE\" FROM bond_description INNER JOIN quotes ON bond_description.\"ISINCODE\"= "\
    "quotes.\"ISIN\" WHERE \"STATUS\"!='Погашен' and \"STATUS\"!='Размещается' and \"HAVEDEFAULT\"='0' AND "\
    "bond_description.\"ISSUERNAME\"="+issuername+" AND quotes.\"CPN_DATE\">="+cpndate+""
    bonds_desc=pd.read_sql_query(query_bonds,conn)
    
    fixed_coup=bonds_desc[['ISINCODE','ISSUERNAME','CPN','CPN_DATE','MAT_DATE','FACEVALUE']][(bonds_desc['COUPONTYPE']=='Постоянный')|(bonds_desc['COUPONTYPE']=='Фиксированный')]
    fixed_coup['CPN'].fillna(0, inplace=True)
    
    import numpy as np
    fixed_coup['CF']=np.where(fixed_coup['CPN_DATE']==fixed_coup['MAT_DATE'], fixed_coup['CPN']+fixed_coup['FACEVALUE'], fixed_coup['CPN'])
    
    fcoup_formatrix=fixed_coup[['ISSUERNAME','ISINCODE','CPN_DATE','CF']]
    
    fcoup_formatrix=fcoup_formatrix.pivot_table(index='ISINCODE', columns='CPN_DATE', values='CF',aggfunc='sum').reset_index().rename_axis(None, axis=1)
    fcoup_formatrix=fcoup_formatrix.fillna(0)

    results={'ISSUERNAME': issuername, 'CPN_DATE': cpndate, 'FMATRIX': fcoup_formatrix }
    return results

In [63]:
# Исполняем функцию (чтобы получилась матрица платежей с учетом всех дат,
# в качестве аргумента для cpndate я указала наименьшую из существующих в документе дат)
login={'user':'postgres', 'database':'postgres', 'host':'localhost', 'port':'5432'}
issuername='\'РЖД\''
cpndate ='\'2009-06-11\''
x=my_fun2(issuername, cpndate, conn)
x['FMATRIX']

,ISINCODE,2009-12-28,2010-06-28,2010-12-27,2011-06-27,2011-12-26,2012-06-25,2012-12-24,2013-06-24,2013-12-23,2014-06-23,2014-12-22,2015-06-22,2015-12-21,2016-04-14
0,RU000A0JQ6S9,0.0,0.0,55.65,108.01,52.36,104.12,101.72,90.6,40.64,40.64,40.64,40.64,81.2784,0.0000
1,RU000A0JVUM4,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0000,57.0433


In [40]:
# данный запрос нужен был для проверки,а также для понимания, у каких компаний есть облигации с постоянным типом купона (чтобы проверить работу функции)
query_bonds=" SELECT DISTINCT bond_description.\"ISINCODE\",bond_description.\"ISSUERNAME\", "\
    "bond_description.\"HAVEOFFER\", bond_description.\"COUPONTYPE\", bond_description.\"SECURITYTYPE\","\
    "quotes.\"BUYBACKDATE\", quotes.\"CPN\", quotes.\"CPN_DATE\", quotes.\"CPN_PERIOD\", quotes.\"FACEVALUE\","\
    "quotes.\"MAT_DATE\" FROM bond_description INNER JOIN quotes ON bond_description.\"ISINCODE\"= "\
    "quotes.\"ISIN\" WHERE \"STATUS\"!='Погашен' and \"STATUS\"!='Размещается' and \"HAVEDEFAULT\"='0' AND bond_description.\"COUPONTYPE\"='Постоянный'"
bonds_desc=pd.read_sql_query(query_bonds,conn)
bonds_desc

,ISINCODE,ISSUERNAME,HAVEOFFER,COUPONTYPE,SECURITYTYPE,BUYBACKDATE,CPN,CPN_DATE,CPN_PERIOD,FACEVALUE,MAT_DATE
0,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,40.6384,2015-12-21,182.0,1000.0,2016-06-20
1,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,40.6400,2013-06-24,182.0,1000.0,2016-06-20
2,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,40.6400,2013-12-23,182.0,1000.0,2016-06-20
3,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,40.6400,2014-06-23,182.0,1000.0,2016-06-20
4,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,40.6400,2014-12-22,182.0,1000.0,2016-06-20
5,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,40.6400,2015-06-22,182.0,1000.0,2016-06-20
6,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,40.6400,2015-12-21,182.0,1000.0,2016-06-20
7,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,49.9600,2012-12-24,182.0,1000.0,2016-06-20
8,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,49.9600,2013-06-24,182.0,1000.0,2016-06-20
9,RU000A0JQ6S9,РЖД,True,Постоянный,Корп,None,51.7600,2012-06-25,182.0,1000.0,2016-06-20
